# BC 5
## Flat ensemble of tiny networks
## Input = sub-region

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from scipy import stats
from sklearn.metrics import accuracy_score

In [2]:
NUM_TN = 30
SUB_REGION_SCALE = 14

#dataset specific parameters
NUM_CLASS = 10

## Data Pre-Processing

In [3]:
#load dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
#Generate cropped train image
X_train_cropped_list = []
y_train_cropped_list = []

for i in range(NUM_TN):
  X_train_cropped = []
  y_train_cropped = []
  for i in range (X_train.shape[0]):
    image_cropped = tf.image.random_crop(X_train[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE])
    image_cropped = np.array(image_cropped)
    X_train_cropped.append(image_cropped)
    y_train_cropped.append(y_train[i])
  X_train_cropped = np.array(X_train_cropped)
  y_train_cropped = np.array(y_train_cropped)

  X_train_cropped_list.append(X_train_cropped)
  y_train_cropped_list.append(y_train_cropped)

In [5]:
#Generate cropped test image
X_test_cropped_list = []
y_test_cropped_list = []

for i in range(NUM_TN):
  X_test_cropped = []
  y_test_cropped = []
  for i in range (X_test.shape[0]):
    image_cropped = tf.image.random_crop(X_test[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE])
    image_cropped = np.array(image_cropped)
    X_test_cropped.append(image_cropped)
    y_test_cropped.append(y_test[i])
  X_test_cropped = np.array(X_test_cropped)
  y_test_cropped = np.array(y_test_cropped)

  X_test_cropped_list.append(X_test_cropped)
  y_test_cropped_list.append(y_test_cropped)

In [6]:
#normalizing dataset
for i in range(NUM_TN):
  X_train_cropped_list[i] = X_train_cropped_list[i]/255
  X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [7]:
#transform to vector
for i in range(NUM_TN):
  X_train_cropped_list[i] = X_train_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE))
  X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE))

In [8]:
#one-hot encoding
for i in range(NUM_TN):
  y_train_cropped_list[i] = to_categorical(y_train_cropped_list[i])
  y_test_cropped_list[i] = to_categorical(y_test_cropped_list[i])

## Model

In [9]:
#build ANN model
ensemble = []
for i in range(NUM_TN):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append(model)

In [10]:
#train model
history = []
for i in range(NUM_TN):
  print('Model %d' %i)
  hist = ensemble[i].fit(X_train_cropped_list[i], y_train_cropped_list[i], validation_data=(X_test_cropped_list[i], y_test_cropped_list[i]), epochs=5, batch_size=128)
  history.append(hist)

Model 0
Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 1.7684 - accuracy: 0.3493 - val_loss: 1.3089 - val_accuracy: 0.5137
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 1.2667 - accuracy: 0.5301 - val_loss: 1.2005 - val_accuracy: 0.5553
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 1.1756 - accuracy: 0.5605 - val_loss: 1.1370 - val_accuracy: 0.5746
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 1.1093 - accuracy: 0.5882 - val_loss: 1.1087 - val_accuracy: 0.5907
Epoch 5/5
469/469 [==============================] - 1s 2ms/step - loss: 1.0850 - accuracy: 0.5981 - val_loss: 1.1051 - val_accuracy: 0.5826
Model 1
Epoch 1/5
469/469 [==============================] - 2s 2ms/step - loss: 1.7528 - accuracy: 0.3519 - val_loss: 1.3141 - val_accuracy: 0.5170
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 1.2797 - accuracy: 0.5264 - val_loss: 1.2041 - val_accuracy: 0.55

## Model Evaluation

In [11]:
# Predict on test image
y_pred_list = []
for i in range(NUM_TN):
  y_pred = ensemble[i].predict(X_test_cropped_list[i])
  y_pred_list.append(y_pred)
    
# Ensemble voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.argmax(np.sum(y_pred_list, axis=0), axis=1)

In [12]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.7356